In [11]:
from collections import defaultdict

data_path = 'data/data.txt'

with open(data_path, 'r') as f:
    lines = f.readlines()

turn_score = defaultdict(lambda: defaultdict(list))
all_scores = []
for line in lines[1:]:
    turn_id, user_utterance, response,  passage_id, passage_txt, score, ptkb, _ = line.split('\t')
    turn_score[turn_id][score].append(passage_id)
    all_scores.append(score)



In [8]:
print(set(all_scores))


{'0', '4', '1', '2', '3'}


In [10]:
from collections import Counter

res = list(Counter(all_scores).items())
print(res)

[('0', 20457), ('1', 2787), ('2', 1803), ('3', 932), ('4', 179)]


In [3]:
turn_binary_count = defaultdict(lambda: defaultdict(int))
turn_binary = defaultdict(lambda: defaultdict(list))


for turn_id in turn_score:
    for score in turn_score[turn_id]:
        if int(score)>=2:
            turn_binary_count[turn_id]['relevant'] += len(turn_score[turn_id][score])
            turn_binary[turn_id]['relevant'] += turn_score[turn_id][score]
        elif int(score)<2:    
            turn_binary_count[turn_id]['irrelevant'] += len(turn_score[turn_id][score])
            turn_binary[turn_id]['irrelevant'] += turn_score[turn_id][score]



In [ ]:
length_1 = 0
for turn_id in turn_binary:
    length_1 += len(turn_binary[turn_id]['relevant'])
    length_1 += len(turn_binary[turn_id]['irrelevant'])
print(length_1)    

In [ ]:
import random
random.seed(10)
data_length = 0

for turn_id in turn_binary:
    turn_binary[turn_id]['irrelevant'] = random.choices(turn_binary[turn_id]['irrelevant'], k=len(turn_binary[turn_id]['relevant']))
    print(turn_binary[turn_id]['relevant'])
    print(turn_binary[turn_id]['irrelevant'])
    print('\n*****************\n')
    data_length += len(turn_binary[turn_id]['irrelevant']) + len(turn_binary[turn_id]['relevant'])


print(data_length)    

test_and_dev_size = int(data_length*0.3)
print(test_and_dev_size)


In [ ]:
Y=0

validation_test_set = defaultdict(list)
validation_set = defaultdict(list)

for turn_id in turn_binary:
    if len(turn_binary[turn_id]['relevant'])>1:
        tmp = turn_binary[turn_id]['relevant'] + turn_binary[turn_id]['irrelevant']
        length_tmp = len(tmp)
        selected_passages = random.choices(tmp, k= int(length_tmp* 0.35) )
        validation_test_set[turn_id] = selected_passages
        
        Y += len(selected_passages)

        if len(selected_passages)>1:
            validation_set[turn_id] = random.choices(selected_passages, k= int(len(selected_passages)* 0.5) )

        
        print(len(validation_test_set[turn_id]))
        print(len(validation_set[turn_id]))
        print('\n*******************\n')

print(Y)        


In [12]:
count_label = defaultdict(int)
all_new_lines = []

for line in lines[1:]:
    lable = 'NONE'
    turn_id, user_utterance, response,  passage_id, passage_txt, score, ptkb, _ = line.split('\t')
    if passage_id in validation_set[turn_id]:
        lable = 'validation'
    elif passage_id in validation_test_set[turn_id]:
        lable = 'test'
    elif (passage_id in turn_binary[turn_id]['relevant']):
        lable = 'train'
    elif (passage_id in turn_binary[turn_id]['irrelevant']):
        lable = 'train'
        
    count_label[lable] += 1
    new_line = '\t'.join([turn_id, user_utterance, response,  passage_id, passage_txt, score, ptkb, lable])+'\n'
    all_new_lines.append(new_line)

print(count_label)    


with open('splitted_data.txt', 'w')    as f:
    f.writelines(all_new_lines)

defaultdict(<class 'int'>, {'NONE': 20680, 'train': 3852, 'validation': 709, 'test': 917})


In [10]:

n = count_label['validation'] + count_label['test']
y = count_label['train']
print(n)
print(n/(n+y))
print(n+y)


1626
0.2968236582694414
5478


In [ ]:
file_name = 'splitted_data.txt'

with open (file_name, 'r') as f:
    lines = f.readlines()

for line in lines: 
    turn_id, user_utterance, response,  passage_id, passage_txt, score, ptkb, lable = line.split('\t')  

   

In [4]:
import pandas as pd
import csv
file_path = "path_to_file"

with open(file_path, 'r') as file:
    reader = csv.reader(file, delimiter='\t')
    df = pd.DataFrame(reader, columns=['turn_id', 'user_utterance', 'response',  'passage_id', 'passage_txt', 'score', 'ptkb', 'lable'])
df.head()   

print(set(list(df['lable'])))

{'test', 'train', 'NONE', 'validation'}


In [13]:
import pandas as pd
import csv
file_path = "splitted_data.txt"

with open(file_path, 'r') as file:
    reader = csv.reader(file, delimiter='\t')
    df = pd.DataFrame(reader, columns=['turn_id', 'user_utterance', 'response',  'passage_id', 'passage_txt', 'score', 'ptkb', 'lable'])
df.head()   

print(set(list(df['lable'])))

{'test', 'train', 'NONE', 'validation'}


In [14]:
print(len(df[df['lable']=='test']))

917


In [15]:
print(len(df[df['lable']=='train']))

3852


In [16]:
print(len(df[df['lable']=='validation']))

709


In [17]:
print(len(df[df['lable']=='NONE']))

20680
